Trey Tuscai and Gordon Doore

Spring 2025

CS 444: Deep Learning

Project 2: Branch Neural Networks

#### Week 2: Residual networks

The focus this week is on the ResNet architecture. You will build several neural networks in the ResNet family and and train them on CIFAR-10 and CIFAR-100.

In [1]:
import numpy as np
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt

plt.style.use(['seaborn-v0_8-colorblind', 'seaborn-v0_8-darkgrid'])
plt.rcParams.update({'font.size': 20})

np.set_printoptions(suppress=True, precision=7)

# Automatically reload your external source code
%load_ext autoreload
%autoreload 2

2025-04-06 16:58:57.263229: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-06 16:59:04.345661: I tensorflow/core/platform/cpu_feature_guard.cc:211] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Task 6: The Residual Block

This task focuses on implementing and testing the **Residual Block** in preparation of creating the first ResNet (**ResNet-8**). 

Much like how Inception Blocks represent the building blocks of Inception Net, stacks of Residual Blocks represent the basis of ResNet. Residual Blocks possess a simpler structure than Inception Blocks — they only contain two parallel branches with fewer layers. Here is a refresher on the structure of the branches:

**Main branch:** sequence of two 2D convolutional layers.

**Residual branch:** the input signal to the Residual Block passes through "as-is", without modification (usually).

Like Inception Block, the output of both branches comes together at the end of the block. However, the branch outputs are SUMMED together rather than being concatenated.


This is the story for most Residual Blocks, however, like most CNNs:
1. the spatial resolution of the activations occasionally decreases
2. the number of conv filters/neurons increases

as we go deeper in a ResNet. Both of these factors tend to change *at the same time* in a small number of Residual Blocks located at various depths of the ResNet. Put another way, the spatial resolution and number of filters tends to remain constant across most successive Residual Blocks and they only changes in a few blocks throughout the net.
1. The decrease in spatial resolution is implemented in these small number of Residual Blocks with a convolutional stride > 1.
2. A 1x1 convolutional layer is needed as the "special sauce" along the residual branch to make the shapes of signals in both branches match (*otherwise they could not be summed!*).

### 6a. Implement and test the Residual Block

The class is in `residual_block.py`.

In [2]:
from residual_block import ResidualBlock

#### Test: `ResidualBlock` Stride 1 (1/2)

In [3]:
# Testing architecture and shapes
# Stride 1
tf.random.set_seed(0)
res1 = ResidualBlock('TestResidualBlock_S1', 7, prev_layer_or_block=None, strides=1)
res1(tf.ones([1, 4, 4, 7]))
print(res1)

2025-04-06 17:00:12.147111: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 20601 MB memory:  -> device: 0, name: NVIDIA L4, pci bus id: 0000:00:03.0, compute capability: 8.9


2025-04-06 17:00:17.751333: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 90400


TestResidualBlock_S1:
	Conv2D layer output(TestResidualBlock_S1/main_3x3conv_2) shape: [1, 4, 4, 7]
	Conv2D layer output(TestResidualBlock_S1/main_3x3conv_1) shape: [1, 4, 4, 7]


The above cell should print:

```
TestResidualBlock_S1:
	Conv2D layer output(TestResidualBlock_S1/main_3x3conv_2) shape: [1, 4, 4, 7]
	Conv2D layer output(TestResidualBlock_S1/main_3x3conv_1) shape: [1, 4, 4, 7]
```

In [4]:
# Test activations
tf.random.set_seed(0)
net_acts1 = res1(tf.random.uniform([2, 4, 4, 7]))
print(f'The shape of the netAct output from the block is {net_acts1.shape} and should be (2, 4, 4, 7)')
print(f'The first few activations are:\n{net_acts1[0,:,:, 0]}')
print('and should be:')
print('''[[0.        0.520823  0.1888617 0.       ]
 [0.        0.7621396 0.1734907 0.8486798]
 [0.        0.6156113 0.4272216 0.       ]
 [0.5561852 0.4888234 1.0138503 0.5533389]]''')

The shape of the netAct output from the block is (2, 4, 4, 7) and should be (2, 4, 4, 7)
The first few activations are:
[[0.        0.5208229 0.1888617 0.       ]
 [0.        0.7621396 0.1734906 0.8486798]
 [0.        0.6156112 0.4272216 0.       ]
 [0.5561852 0.4888234 1.0138503 0.5533389]]
and should be:
[[0.        0.520823  0.1888617 0.       ]
 [0.        0.7621396 0.1734907 0.8486798]
 [0.        0.6156113 0.4272216 0.       ]
 [0.5561852 0.4888234 1.0138503 0.5533389]]


#### Test: `ResidualBlock` Stride 2 (2/2)

In [5]:
# Testing architecture and shapes
# Stride 2
tf.random.set_seed(0)
res2 = ResidualBlock('TestResidualBlock_S2', 5, prev_layer_or_block=None, strides=2)
res2(tf.ones([1, 6, 6, 5]))
print(res2)

TestResidualBlock_S2:
	Conv2D layer output(TestResidualBlock_S2/main_3x3conv_2) shape: [1, 3, 3, 5]
	Conv2D layer output(TestResidualBlock_S2/main_3x3conv_1) shape: [1, 3, 3, 5]
	-->Conv2D1x1 layer output(TestResidualBlock_S2/skip_conv1x1) shape: [1, 3, 3, 5]-->


The above cell should print:

```
TestResidualBlock_S2:
	Conv2D layer output(TestResidualBlock_S2/main_3x3conv_2) shape: [1, 3, 3, 5]
	Conv2D layer output(TestResidualBlock_S2/main_3x3conv_1) shape: [1, 3, 3, 5]
	-->Conv2D1x1 layer output(TestResidualBlock_S2/skip_conv1x1) shape: [1, 3, 3, 5]-->
```

*The layer with the --> is the residual branch.*

In [6]:
# Test activations
tf.random.set_seed(0)
net_acts2 = res2(tf.random.uniform([3, 6, 6, 5]))
print(f'The shape of the netAct output from the block is {net_acts2.shape} and should be (3, 3, 3, 5)')
print(f'The first few activations are:\n{net_acts2[0,:,:, :]}')
print('and should be:')
print('''[[[0.2404823 0.        0.        0.2851936 0.       ]
  [0.        0.        0.        0.1339086 0.6898913]
  [0.        0.        0.        0.4596353 0.2781557]]

 [[0.        0.        0.        0.6591434 1.3703969]
  [0.2665227 0.        0.        0.9614864 0.       ]
  [0.        0.        0.        0.3844326 0.7111533]]

 [[0.0933782 0.        0.        0.1378801 0.3006183]
  [0.1873689 0.        0.        0.4464224 1.1067129]
  [0.        0.        0.        0.7910071 0.345379 ]]]''')

The shape of the netAct output from the block is (3, 3, 3, 5) and should be (3, 3, 3, 5)
The first few activations are:
[[[0.2404822 0.        0.        0.2851935 0.       ]
  [0.        0.        0.        0.1339087 0.6898913]
  [0.        0.        0.        0.4596352 0.2781556]]

 [[0.        0.        0.        0.6591434 1.3703969]
  [0.2665228 0.        0.        0.9614864 0.       ]
  [0.        0.        0.        0.3844326 0.7111532]]

 [[0.0933783 0.        0.        0.1378801 0.3006182]
  [0.1873689 0.        0.        0.4464225 1.106713 ]
  [0.        0.        0.        0.7910072 0.345379 ]]]
and should be:
[[[0.2404823 0.        0.        0.2851936 0.       ]
  [0.        0.        0.        0.1339086 0.6898913]
  [0.        0.        0.        0.4596353 0.2781557]]

 [[0.        0.        0.        0.6591434 1.3703969]
  [0.2665227 0.        0.        0.9614864 0.       ]
  [0.        0.        0.        0.3844326 0.7111533]]

 [[0.0933782 0.        0.        0.1378801 0.

## Task 7: ResNet-8

Assemble the Residual Blocks and several other layers to build ResNet-8:

Conv2D → ResidualBlock → ResidualBlock → ResidualBlock → GlobalAveragePooling2D → Dense

After an overfit test to help check whether the network is working, you will train the network on both CIFAR-10 and CIFAR-100.

In [7]:
from resnets import ResNet8

### 7a. Build ResNet-8

Implement the following classes in `resnets.py`:
1. `ResNet`: Parent class of all specific ResNets (e.g. ResNet-8, ResNet-18, etc.). Having this class helps reduce code size/duplication because the forward pass thru all ResNets is exactly the same!
2. `ResNet8`: Assemble the first (*and smallest*) net in the family!

#### Test: `ResNet8` architecture and shapes

In [8]:
res8 = ResNet8(C=3, input_feats_shape=(32, 32, 3))
res8.compile()

---------------------------------------------------------------------------
Dense layer output(Output) shape: [1, 3]
Global Avg Pooling 2D layer output(GlobalAveragePool2D) shape: [1, 128]
ResidualBlock_3:
	Conv2D layer output(ResidualBlock_3/main_3x3conv_2) shape: [1, 8, 8, 128]
	Conv2D layer output(ResidualBlock_3/main_3x3conv_1) shape: [1, 8, 8, 128]
	-->Conv2D1x1 layer output(ResidualBlock_3/skip_conv1x1) shape: [1, 8, 8, 128]-->
ResidualBlock_2:
	Conv2D layer output(ResidualBlock_2/main_3x3conv_2) shape: [1, 16, 16, 64]
	Conv2D layer output(ResidualBlock_2/main_3x3conv_1) shape: [1, 16, 16, 64]
	-->Conv2D1x1 layer output(ResidualBlock_2/skip_conv1x1) shape: [1, 16, 16, 64]-->
ResidualBlock_1:
	Conv2D layer output(ResidualBlock_1/main_3x3conv_2) shape: [1, 32, 32, 32]
	Conv2D layer output(ResidualBlock_1/main_3x3conv_1) shape: [1, 32, 32, 32]
Conv2D layer output(Conv2D_1) shape: [1, 32, 32, 32]


The above cell should print:

```
---------------------------------------------------------------------------
Dense layer output(Output) shape: [1, 3]
Global Avg Pooling 2D layer output(GlobalAvgPool2D) shape: [1, 128]
ResidualBlock_3:
	Conv2D layer output(ResidualBlock_3/main_3x3conv_2) shape: [1, 8, 8, 128]
	Conv2D layer output(ResidualBlock_3/main_3x3conv_1) shape: [1, 8, 8, 128]
	-->Conv2D1x1 layer output(ResidualBlock_3/skip_conv1x1) shape: [1, 8, 8, 128]-->
ResidualBlock_2:
	Conv2D layer output(ResidualBlock_2/main_3x3conv_2) shape: [1, 16, 16, 64]
	Conv2D layer output(ResidualBlock_2/main_3x3conv_1) shape: [1, 16, 16, 64]
	-->Conv2D1x1 layer output(ResidualBlock_2/skip_conv1x1) shape: [1, 16, 16, 64]-->
ResidualBlock_1:
	Conv2D layer output(ResidualBlock_1/main_3x3conv_2) shape: [1, 32, 32, 32]
	Conv2D layer output(ResidualBlock_1/main_3x3conv_1) shape: [1, 32, 32, 32]
Conv2D layer output(Conv2D_1) shape: [1, 32, 32, 32]
```

### 7b. CIFAR-10 overfit test

In the cell below, import CIFAR-10 and reproduce our usual overfit protocol:
1. Create a dev set from the 1st 500 training CIFAR-10 samples.
2. Train your net on the dev set for `80` epochs (turn off early stopping for this test). *Do not use any regularization.* 

Your training loss should start out at ~2.3 after the first epoch and rapidly plummet to 0.01 or less by about 70 epochs.

**Note:** If you coded `fit` to assume there will always be a validation set present, no problem, just plug in the dev set for both the train and val sets.

In [9]:
from datasets import get_dataset

In [10]:
x_train, y_train, x_val, y_val, x_test, y_test, classnames = get_dataset('cifar10')
x_dev = x_train[:500]
y_dev = y_train[:500]

In [11]:
tf.keras.backend.clear_session()
tf.random.set_seed(0)

# YOUR CODE HERE
model = ResNet8(10, (32,32,3), reg = 0)
model.compile(optimizer='adamw')
model.fit(x_dev, y_dev, x_dev, y_dev, max_epochs = 80, val_every = 1, verbose = True)

---------------------------------------------------------------------------
Dense layer output(Output) shape: [1, 10]
Global Avg Pooling 2D layer output(GlobalAveragePool2D) shape: [1, 128]
ResidualBlock_3:
	Conv2D layer output(ResidualBlock_3/main_3x3conv_2) shape: [1, 8, 8, 128]
	Conv2D layer output(ResidualBlock_3/main_3x3conv_1) shape: [1, 8, 8, 128]
	-->Conv2D1x1 layer output(ResidualBlock_3/skip_conv1x1) shape: [1, 8, 8, 128]-->
ResidualBlock_2:
	Conv2D layer output(ResidualBlock_2/main_3x3conv_2) shape: [1, 16, 16, 64]
	Conv2D layer output(ResidualBlock_2/main_3x3conv_1) shape: [1, 16, 16, 64]
	-->Conv2D1x1 layer output(ResidualBlock_2/skip_conv1x1) shape: [1, 16, 16, 64]-->
ResidualBlock_1:
	Conv2D layer output(ResidualBlock_1/main_3x3conv_2) shape: [1, 32, 32, 32]
	Conv2D layer output(ResidualBlock_1/main_3x3conv_1) shape: [1, 32, 32, 32]
Conv2D layer output(Conv2D_1) shape: [1, 32, 32, 32]


I0000 00:00:1743958870.215802     584 service.cc:145] XLA service 0x799e0e54e640 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1743958870.215851     584 service.cc:153]   StreamExecutor device (0): NVIDIA L4, Compute Capability 8.9


2025-04-06 17:01:10.628313: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


I0000 00:00:1743958871.801254     584 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Epoch 1: Training Loss = 2.5854, Validation Loss = 2.3263, Validation Accuracy = 0.1473
Epoch 1/80 took 13.3567 seconds
Epoch 2: Training Loss = 2.2497, Validation Loss = 2.1903, Validation Accuracy = 0.1920
Epoch 2/80 took 0.1009 seconds
Epoch 3: Training Loss = 2.1547, Validation Loss = 2.1454, Validation Accuracy = 0.2188
Epoch 3/80 took 0.0528 seconds


...

Epoch 68: Training Loss = 0.1607, Validation Loss = 0.1625, Validation Accuracy = 0.9621
Epoch 68/80 took 0.0529 seconds
Epoch 69: Training Loss = 0.1851, Validation Loss = 0.1756, Validation Accuracy = 0.9598
Epoch 69/80 took 0.0518 seconds
Epoch 70: Training Loss = 0.1694, Validation Loss = 0.2109, Validation Accuracy = 0.9308
Epoch 70/80 took 0.0522 seconds
Epoch 71: Training Loss = 0.1941, Validation Loss = 0.1960, Validation Accuracy = 0.9464
Epoch 71/80 took 0.0520 seconds


Epoch 72: Training Loss = 0.1501, Validation Loss = 0.1372, Validation Accuracy = 0.9732
Epoch 72/80 took 0.0525 seconds
Epoch 73: Training Loss = 0.1382, Validation Loss = 0.1261, Validation Accuracy = 0.9754
Epoch 73/80 took 0.0515 seconds
Epoch 74: Training Loss = 0.1231, Validation Loss = 0.1195, Validation Accuracy = 0.9732
Epoch 74/80 took 0.0521 seconds
Epoch 75: Training Loss = 0.1503, Validation Loss = 0.1631, Validation Accuracy = 0.9487
Epoch 75/80 took 0.0520 seconds


Epoch 76: Training Loss = 0.1663, Validation Loss = 0.1322, Validation Accuracy = 0.9754
Epoch 76/80 took 0.0541 seconds
Epoch 77: Training Loss = 0.1120, Validation Loss = 0.1144, Validation Accuracy = 0.9777
Epoch 77/80 took 0.0519 seconds
Epoch 78: Training Loss = 0.1122, Validation Loss = 0.0961, Validation Accuracy = 0.9888
Epoch 78/80 took 0.0517 seconds
Epoch 79: Training Loss = 0.0958, Validation Loss = 0.0852, Validation Accuracy = 0.9844
Epoch 79/80 took 0.0515 seconds


Epoch 80: Training Loss = 0.0722, Validation Loss = 0.0679, Validation Accuracy = 0.9933
Epoch 80/80 took 0.0529 seconds
Finished training after 80 epochs!


([2.5854297,
  2.2497468,
  2.1547399,
  2.1165285,
  2.0975192,
  2.0260031,
  2.0233045,
  2.0125296,
  1.93002,
  1.9201531,
  1.8509347,
  1.889932,
  1.8792644,
  1.7786869,
  1.7516723,
  1.7173429,
  1.697966,
  1.5990942,
  1.5918684,
  1.6312492,
  1.6032494,
  1.545558,
  1.5533642,
  1.4297822,
  1.4482217,
  1.301119,
  1.2720656,
  1.3518972,
  1.1772319,
  1.1537479,
  1.1814579,
  1.1633275,
  1.098788,
  1.0707209,
  0.95325476,
  1.0494305,
  1.0817847,
  0.98039925,
  0.9782729,
  0.9178203,
  0.87792253,
  0.95758,
  0.76955914,
  0.8141868,
  0.69339216,
  0.6646242,
  0.7035171,
  0.7651026,
  0.8157638,
  0.7384411,
  0.7697588,
  0.74597955,
  0.6499058,
  0.6051854,
  0.61780906,
  0.56082433,
  0.53941995,
  0.5118757,
  0.43944666,
  0.3429498,
  0.3608862,
  0.32623875,
  0.28941652,
  0.24795817,
  0.24768978,
  0.1816405,
  0.22513843,
  0.16066617,
  0.185137,
  0.16936228,
  0.19414638,
  0.15012631,
  0.1381653,
  0.12310675,
  0.15025103,
  0.16629529,


### 7c. Train ResNet-8 on CIFAR-10

Repeat our usual training and evaluation protocol:
1. Train ResNet-8 on CIFAR-10. Use regularization strength of `1.5`, a patience of `15`, learning rate patience of `4`, and keep the rest of the hyperparameters to their defaults.
2. Print the test accuracy.

If everything is working as expected, you should get a test accuracy in the 80s.

In [12]:
tf.keras.backend.clear_session()
tf.random.set_seed(0)

# YOUR CODE HERE
model = ResNet8(10, (32,32,3), reg = 1.5)
model.compile(optimizer='adamw')
model.fit(x_train, y_train, x_val, y_val, val_every = 1, verbose = True, patience=15, lr_patience=4)
print(f"Test acc: {model.evaluate(x_test, y_test)[0]}")

---------------------------------------------------------------------------
Dense layer output(Output) shape: [1, 10]
Global Avg Pooling 2D layer output(GlobalAveragePool2D) shape: [1, 128]
ResidualBlock_3:
	Conv2D layer output(ResidualBlock_3/main_3x3conv_2) shape: [1, 8, 8, 128]
	Conv2D layer output(ResidualBlock_3/main_3x3conv_1) shape: [1, 8, 8, 128]
	-->Conv2D1x1 layer output(ResidualBlock_3/skip_conv1x1) shape: [1, 8, 8, 128]-->
ResidualBlock_2:
	Conv2D layer output(ResidualBlock_2/main_3x3conv_2) shape: [1, 16, 16, 64]
	Conv2D layer output(ResidualBlock_2/main_3x3conv_1) shape: [1, 16, 16, 64]
	-->Conv2D1x1 layer output(ResidualBlock_2/skip_conv1x1) shape: [1, 16, 16, 64]-->
ResidualBlock_1:
	Conv2D layer output(ResidualBlock_1/main_3x3conv_2) shape: [1, 32, 32, 32]
	Conv2D layer output(ResidualBlock_1/main_3x3conv_1) shape: [1, 32, 32, 32]
Conv2D layer output(Conv2D_1) shape: [1, 32, 32, 32]


Epoch 1: Training Loss = 1.7492, Validation Loss = 1.5184, Validation Accuracy = 0.4453
Epoch 1/10000 took 6.6293 seconds


Epoch 2: Training Loss = 1.4277, Validation Loss = 1.3107, Validation Accuracy = 0.5226
Epoch 2/10000 took 3.1285 seconds


Epoch 3: Training Loss = 1.3299, Validation Loss = 1.2617, Validation Accuracy = 0.5461
Epoch 3/10000 took 3.1370 seconds


Epoch 4: Training Loss = 1.2878, Validation Loss = 1.1990, Validation Accuracy = 0.5723
Epoch 4/10000 took 3.1521 seconds


...

Epoch 99: Training Loss = 1.0364, Validation Loss = 1.0288, Validation Accuracy = 0.6410
Epoch 99/10000 took 3.2124 seconds


Epoch 100: Training Loss = 1.0368, Validation Loss = 1.0300, Validation Accuracy = 0.6420
Epoch 100/10000 took 3.2166 seconds


Epoch 101: Training Loss = 1.0380, Validation Loss = 1.0294, Validation Accuracy = 0.6420
Epoch 101/10000 took 3.2096 seconds


Current lr= 4.882813e-07 Updated lr= 2.4414064e-07
Epoch 102: Training Loss = 1.0346, Validation Loss = 1.0290, Validation Accuracy = 0.6410
Early stopping triggered at epoch 102
Finished training after 102 epochs!


Test acc: 0.6266025900840759


### 7d. Train ResNet-8 on CIFAR-100

Repeat what you did with CIFAR-10, but this time with CIFAR-100.

The test accuracy that you achieve should be better than chance, but should NOT be satisfying.

In [37]:
tf.keras.backend.clear_session()
tf.random.set_seed(0)

x100_train, y100_train, x100_val, y100_val, x100_test, y100_test, classnames = get_dataset('cifar100')

model = ResNet8(100, (32,32,3), reg = 1.5)
model.compile(optimizer='adamw')
model.fit(x100_train, y100_train, x100_val, y100_val, val_every = 1, verbose = True, patience=15, lr_patience=4)
print(f"Test acc: {model.evaluate(x100_test, y100_test)[0]}")

---------------------------------------------------------------------------
Dense layer output(Output) shape: [1, 100]
Global Avg Pooling 2D layer output(GlobalAveragePool2D) shape: [1, 128]
ResidualBlock_3:
	Conv2D layer output(ResidualBlock_3/main_3x3conv_2) shape: [1, 8, 8, 128]
	Conv2D layer output(ResidualBlock_3/main_3x3conv_1) shape: [1, 8, 8, 128]
	-->Conv2D1x1 layer output(ResidualBlock_3/skip_conv1x1) shape: [1, 8, 8, 128]-->
ResidualBlock_2:
	Conv2D layer output(ResidualBlock_2/main_3x3conv_2) shape: [1, 16, 16, 64]
	Conv2D layer output(ResidualBlock_2/main_3x3conv_1) shape: [1, 16, 16, 64]
	-->Conv2D1x1 layer output(ResidualBlock_2/skip_conv1x1) shape: [1, 16, 16, 64]-->
ResidualBlock_1:
	Conv2D layer output(ResidualBlock_1/main_3x3conv_2) shape: [1, 32, 32, 32]
	Conv2D layer output(ResidualBlock_1/main_3x3conv_1) shape: [1, 32, 32, 32]
Conv2D layer output(Conv2D_1) shape: [1, 32, 32, 32]


Epoch 1: Training Loss = 4.0810, Validation Loss = 3.8104, Validation Accuracy = 0.1122
Epoch 1/10000 took 9.1026 seconds


Epoch 2: Training Loss = 3.5850, Validation Loss = 3.4828, Validation Accuracy = 0.1693
Epoch 2/10000 took 2.5555 seconds


Epoch 3: Training Loss = 3.3285, Validation Loss = 3.4021, Validation Accuracy = 0.1847
Epoch 3/10000 took 2.5625 seconds


Epoch 4: Training Loss = 3.1959, Validation Loss = 3.2161, Validation Accuracy = 0.2179
Epoch 4/10000 took 2.5911 seconds


...

Epoch 107: Training Loss = 2.2542, Validation Loss = 2.4857, Validation Accuracy = 0.3688
Epoch 107/10000 took 2.5775 seconds


Epoch 108: Training Loss = 2.2704, Validation Loss = 2.4856, Validation Accuracy = 0.3692
Epoch 108/10000 took 2.5687 seconds


Epoch 109: Training Loss = 2.2665, Validation Loss = 2.4852, Validation Accuracy = 0.3696
Epoch 109/10000 took 2.5918 seconds


Epoch 110: Training Loss = 2.2669, Validation Loss = 2.4854, Validation Accuracy = 0.3686
Early stopping triggered at epoch 110
Finished training after 110 epochs!


Test acc: 0.3891226053237915


### 7e. Questions

**Question 3:** Compare your ResNet-8 with Inception Net with respect to CIFAR-10 test accuracy, runtime (per epoch), and the train/val loss progression throughout training. 

**Question 4:** How did ResNet-8 do on at CIFAR-100 test set classification compared to Inception Net?

**Answer 3:**

**Answer 4:**

## Task 8: ResNet-18

ResNet is an incredibly flexible/extensible neural network architecture. To get a better sense of this, let's build a deeper ResNet then train it on CIFAR-100.

### 8a. Stacking multiple Residual Blocks together in sequence

In ResNet-8, the spatial resolution/number of filters changed in every residual block. In deeper ResNets, this is not usually the case — there is a "string"/sequence of Residual Blocks with the SAME resolution and filter count stacked together after the change occurs.

To streamline the process of stacking multiple Residual Blocks with the same hyperparameters together, write the `stack_residualblocks` function in `resnets.py`. This should save you lots of copy-pasting and/or typing!

In [25]:
from resnets import stack_residualblocks

#### Test: `stack_residualblocks`

In [26]:
for i in range(4):
    test_stack = stack_residualblocks('TestStack', 4, i+1, prev_layer_or_block=None, first_block_stride=1)
    print(f'There are {len(test_stack)} blocks in the residual stack. There should be {i+1}.')

strides_in_stack = [block.strides for block in test_stack]
print(f'The strides in each block are: {strides_in_stack}. They should be [1, 1, 1, 1]')

test_stack = stack_residualblocks('TestStack', 4, 3, prev_layer_or_block=None, first_block_stride=2)
strides_in_stack = [block.strides for block in test_stack]
print(f'The strides in each block are: {strides_in_stack}. They should be [2, 1, 1, 1]')


There are 1 blocks in the residual stack. There should be 1.
There are 2 blocks in the residual stack. There should be 2.
There are 3 blocks in the residual stack. There should be 3.
There are 4 blocks in the residual stack. There should be 4.
The strides in each block are: [1, 1, 1, 1]. They should be [1, 1, 1, 1]
The strides in each block are: [2, 1, 1]. They should be [2, 1, 1, 1]


In [27]:
print('The blocks are:')
for block in test_stack:
    print(block)

The blocks are:
TestStack/block_1:
	Conv2D layer output(TestStack/block_1/main_3x3conv_2) shape: None
	Conv2D layer output(TestStack/block_1/main_3x3conv_1) shape: None
	-->Conv2D1x1 layer output(TestStack/block_1/skip_conv1x1) shape: None-->
TestStack/block_2:
	Conv2D layer output(TestStack/block_2/main_3x3conv_2) shape: None
	Conv2D layer output(TestStack/block_2/main_3x3conv_1) shape: None
TestStack/block_3:
	Conv2D layer output(TestStack/block_3/main_3x3conv_2) shape: None
	Conv2D layer output(TestStack/block_3/main_3x3conv_1) shape: None


The above should print:

```
The blocks are:
TestStack/block_1:
	Conv2D layer output(TestStack/block_1/main_3x3conv_2) shape: None
	Conv2D layer output(TestStack/block_1/main_3x3conv_1) shape: None
	-->Conv2D1x1 layer output(TestStack/block_1/skip_conv1x1) shape: None-->
TestStack/block_2:
	Conv2D layer output(TestStack/block_2/main_3x3conv_2) shape: None
	Conv2D layer output(TestStack/block_2/main_3x3conv_1) shape: None
TestStack/block_3:
	Conv2D layer output(TestStack/block_3/main_3x3conv_2) shape: None
	Conv2D layer output(TestStack/block_3/main_3x3conv_1) shape: None
```

### 8b. Build ResNet-18

Implement the `ResNet18` class in `resnets.py`.

In [29]:
from resnets import ResNet18

#### Test: `ResNet18`

In [30]:
res18 = ResNet18(C=4, input_feats_shape=(32, 32, 3))
res18.compile()

---------------------------------------------------------------------------
Dense layer output(Output) shape: [1, 4]
Global Avg Pooling 2D layer output(GlobalAveragePool2D) shape: [1, 512]
stack_4/block_2:
	Conv2D layer output(stack_4/block_2/main_3x3conv_2) shape: [1, 4, 4, 512]
	Conv2D layer output(stack_4/block_2/main_3x3conv_1) shape: [1, 4, 4, 512]
stack_4/block_1:
	Conv2D layer output(stack_4/block_1/main_3x3conv_2) shape: [1, 4, 4, 512]
	Conv2D layer output(stack_4/block_1/main_3x3conv_1) shape: [1, 4, 4, 512]
	-->Conv2D1x1 layer output(stack_4/block_1/skip_conv1x1) shape: [1, 4, 4, 512]-->
stack_3/block_2:
	Conv2D layer output(stack_3/block_2/main_3x3conv_2) shape: [1, 8, 8, 256]
	Conv2D layer output(stack_3/block_2/main_3x3conv_1) shape: [1, 8, 8, 256]
stack_3/block_1:
	Conv2D layer output(stack_3/block_1/main_3x3conv_2) shape: [1, 8, 8, 256]
	Conv2D layer output(stack_3/block_1/main_3x3conv_1) shape: [1, 8, 8, 256]
	-->Conv2D1x1 layer output(stack_3/block_1/skip_conv1x1) shap

The above cell should print:

```
---------------------------------------------------------------------------
Dense layer output(Output) shape: [1, 4]
Global Avg Pooling 2D layer output(GlobalAvgPool2D) shape: [1, 512]
stack4/block_2:
	Conv2D layer output(stack4/block_2/main_3x3conv_2) shape: [1, 4, 4, 512]
	Conv2D layer output(stack4/block_2/main_3x3conv_1) shape: [1, 4, 4, 512]
stack4/block_1:
	Conv2D layer output(stack4/block_1/main_3x3conv_2) shape: [1, 4, 4, 512]
	Conv2D layer output(stack4/block_1/main_3x3conv_1) shape: [1, 4, 4, 512]
	-->Conv2D1x1 layer output(stack4/block_1/skip_conv1x1) shape: [1, 4, 4, 512]-->
stack3/block_2:
	Conv2D layer output(stack3/block_2/main_3x3conv_2) shape: [1, 8, 8, 256]
	Conv2D layer output(stack3/block_2/main_3x3conv_1) shape: [1, 8, 8, 256]
stack3/block_1:
	Conv2D layer output(stack3/block_1/main_3x3conv_2) shape: [1, 8, 8, 256]
	Conv2D layer output(stack3/block_1/main_3x3conv_1) shape: [1, 8, 8, 256]
	-->Conv2D1x1 layer output(stack3/block_1/skip_conv1x1) shape: [1, 8, 8, 256]-->
stack2/block_2:
	Conv2D layer output(stack2/block_2/main_3x3conv_2) shape: [1, 16, 16, 128]
	Conv2D layer output(stack2/block_2/main_3x3conv_1) shape: [1, 16, 16, 128]
stack2/block_1:
	Conv2D layer output(stack2/block_1/main_3x3conv_2) shape: [1, 16, 16, 128]
	Conv2D layer output(stack2/block_1/main_3x3conv_1) shape: [1, 16, 16, 128]
	-->Conv2D1x1 layer output(stack2/block_1/skip_conv1x1) shape: [1, 16, 16, 128]-->
stack1/block_2:
	Conv2D layer output(stack1/block_2/main_3x3conv_2) shape: [1, 32, 32, 64]
	Conv2D layer output(stack1/block_2/main_3x3conv_1) shape: [1, 32, 32, 64]
stack1/block_1:
	Conv2D layer output(stack1/block_1/main_3x3conv_2) shape: [1, 32, 32, 64]
	Conv2D layer output(stack1/block_1/main_3x3conv_1) shape: [1, 32, 32, 64]
Conv2D layer output(Conv2D_1) shape: [1, 32, 32, 64]
```

### 8c. Overfit ResNet-18 on CIFAR-100 dev set

Perform the usual overfitting protocol to test out your ResNet-18. However, this time use the 1st 500 samples of CIFAR-100 rather than CIFAR-10 to conduct the test.

In the cell below, import CIFAR-100 and reproduce our usual overfit protocol:
1. Create a dev set from the 1st 500 training CIFAR-100 samples.
2. Train your net on the dev set for `80` epochs (turn off early stopping for this test). *Do not use any regularization.* 

Your training loss should start out at ~4.7 after the first epoch and rapidly plummet to 0.01 or less after about 30 epochs.

In [39]:
x100_train, y100_train, x100_val, y100_val, x100_test, y100_test, classnames100 = get_dataset('cifar100')
x100_dev = x100_train[:500]
y100_dev = y100_train[:500]

In [40]:
tf.keras.backend.clear_session()
tf.random.set_seed(0)

# YOUR CODE HERE
model = ResNet18(100, (32,32,3), reg = 0)
model.compile(optimizer='adamw')
model.fit(x100_dev, y100_dev, x100_dev, y100_dev, max_epochs = 80, val_every = 1, verbose = True)

---------------------------------------------------------------------------
Dense layer output(Output) shape: [1, 100]
Global Avg Pooling 2D layer output(GlobalAveragePool2D) shape: [1, 512]
stack_4/block_2:
	Conv2D layer output(stack_4/block_2/main_3x3conv_2) shape: [1, 4, 4, 512]
	Conv2D layer output(stack_4/block_2/main_3x3conv_1) shape: [1, 4, 4, 512]
stack_4/block_1:
	Conv2D layer output(stack_4/block_1/main_3x3conv_2) shape: [1, 4, 4, 512]
	Conv2D layer output(stack_4/block_1/main_3x3conv_1) shape: [1, 4, 4, 512]
	-->Conv2D1x1 layer output(stack_4/block_1/skip_conv1x1) shape: [1, 4, 4, 512]-->
stack_3/block_2:
	Conv2D layer output(stack_3/block_2/main_3x3conv_2) shape: [1, 8, 8, 256]
	Conv2D layer output(stack_3/block_2/main_3x3conv_1) shape: [1, 8, 8, 256]
stack_3/block_1:
	Conv2D layer output(stack_3/block_1/main_3x3conv_2) shape: [1, 8, 8, 256]
	Conv2D layer output(stack_3/block_1/main_3x3conv_1) shape: [1, 8, 8, 256]
	-->Conv2D1x1 layer output(stack_3/block_1/skip_conv1x1) sh

Epoch 1: Training Loss = 7.7652, Validation Loss = 4.6127, Validation Accuracy = 0.0089
Epoch 1/80 took 8.3178 seconds


Epoch 2: Training Loss = 4.6071, Validation Loss = 4.5911, Validation Accuracy = 0.0089
Epoch 2/80 took 0.2087 seconds


Epoch 3: Training Loss = 4.5924, Validation Loss = 4.5750, Validation Accuracy = 0.0089
Epoch 3/80 took 0.2101 seconds


Epoch 4: Training Loss = 4.5622, Validation Loss = 4.5537, Validation Accuracy = 0.0134
Epoch 4/80 took 0.2059 seconds


...

Epoch 77: Training Loss = 0.0789, Validation Loss = 0.1563, Validation Accuracy = 0.9710
Epoch 77/80 took 0.2160 seconds


Epoch 78: Training Loss = 0.1551, Validation Loss = 0.3448, Validation Accuracy = 0.9420
Epoch 78/80 took 0.2127 seconds


Epoch 79: Training Loss = 0.1381, Validation Loss = 0.2727, Validation Accuracy = 0.9598
Epoch 79/80 took 0.2098 seconds


Epoch 80: Training Loss = 0.2125, Validation Loss = 0.1021, Validation Accuracy = 0.9777
Epoch 80/80 took 0.2154 seconds
Finished training after 80 epochs!


([7.7651725,
  4.6070657,
  4.592352,
  4.562155,
  4.5742607,
  4.540709,
  4.5136595,
  4.5531416,
  4.529939,
  4.514531,
  4.501354,
  4.4903398,
  4.5405436,
  4.5017595,
  4.505933,
  4.53881,
  4.505358,
  4.506524,
  4.486269,
  4.4880586,
  4.497943,
  4.450206,
  4.426097,
  4.419592,
  4.341523,
  4.4022803,
  4.3482614,
  4.345825,
  4.2823424,
  4.2886295,
  4.155503,
  4.185897,
  4.155451,
  4.145169,
  4.05106,
  3.9292145,
  3.8931155,
  3.7422056,
  3.6206708,
  3.461061,
  3.413474,
  3.1410463,
  3.1140924,
  2.872694,
  2.7758555,
  2.4953969,
  2.2750418,
  2.1306148,
  1.801698,
  1.376468,
  1.361461,
  1.0854856,
  1.0065608,
  0.77844626,
  0.64682215,
  0.5809196,
  0.5030327,
  0.4738898,
  0.27428418,
  0.260916,
  0.22176293,
  0.24809341,
  0.22025491,
  0.16066559,
  0.1324631,
  0.08172591,
  0.12648809,
  0.0875947,
  0.049269788,
  0.106942065,
  0.10053298,
  0.07531144,
  0.08623174,
  0.09335752,
  0.08647495,
  0.08121104,
  0.07888458,
  0.155088

### 8d. Train ResNet-18 on CIFAR-100

In the cell below, train your ResNet-18 on CIFAR-100. Print out the test set after training concludes.

Use regularization strength of `1.5`, a patience of `15`, learning rate patience of `4`, and keep the rest of the hyperparameters to their defaults.

In [41]:
tf.keras.backend.clear_session()
tf.random.set_seed(0)

model = ResNet18(100, (32,32,3), reg = 1.5)
model.compile(optimizer='adamw')
model.fit(x100_train, y100_train, x100_val, y100_val, val_every = 1, verbose = True, patience=15, lr_patience=4)
print(f"Test acc: {model.evaluate(x100_test, y100_test)[0]}")

---------------------------------------------------------------------------
Dense layer output(Output) shape: [1, 100]
Global Avg Pooling 2D layer output(GlobalAveragePool2D) shape: [1, 512]
stack_4/block_2:
	Conv2D layer output(stack_4/block_2/main_3x3conv_2) shape: [1, 4, 4, 512]
	Conv2D layer output(stack_4/block_2/main_3x3conv_1) shape: [1, 4, 4, 512]
stack_4/block_1:
	Conv2D layer output(stack_4/block_1/main_3x3conv_2) shape: [1, 4, 4, 512]
	Conv2D layer output(stack_4/block_1/main_3x3conv_1) shape: [1, 4, 4, 512]
	-->Conv2D1x1 layer output(stack_4/block_1/skip_conv1x1) shape: [1, 4, 4, 512]-->
stack_3/block_2:
	Conv2D layer output(stack_3/block_2/main_3x3conv_2) shape: [1, 8, 8, 256]
	Conv2D layer output(stack_3/block_2/main_3x3conv_1) shape: [1, 8, 8, 256]
stack_3/block_1:
	Conv2D layer output(stack_3/block_1/main_3x3conv_2) shape: [1, 8, 8, 256]
	Conv2D layer output(stack_3/block_1/main_3x3conv_1) shape: [1, 8, 8, 256]
	-->Conv2D1x1 layer output(stack_3/block_1/skip_conv1x1) sh

Epoch 1: Training Loss = 4.4807, Validation Loss = 4.3247, Validation Accuracy = 0.0310
Epoch 1/10000 took 22.5567 seconds


Epoch 2: Training Loss = 4.0047, Validation Loss = 3.8745, Validation Accuracy = 0.0857
Epoch 2/10000 took 15.0865 seconds


Epoch 3: Training Loss = 3.6677, Validation Loss = 3.5173, Validation Accuracy = 0.1434
Epoch 3/10000 took 14.9365 seconds


Epoch 4: Training Loss = 3.3394, Validation Loss = 3.3380, Validation Accuracy = 0.1863
Epoch 4/10000 took 14.8153 seconds


...

Epoch 46: Training Loss = 0.4624, Validation Loss = 2.3012, Validation Accuracy = 0.4988
Epoch 46/10000 took 15.1659 seconds


Epoch 47: Training Loss = 0.4267, Validation Loss = 2.3198, Validation Accuracy = 0.5164
Epoch 47/10000 took 15.1437 seconds


Current lr= 6.25e-05 Updated lr= 3.125e-05
Epoch 48: Training Loss = 0.4138, Validation Loss = 2.3522, Validation Accuracy = 0.5230
Epoch 48/10000 took 15.1525 seconds


Epoch 49: Training Loss = 0.2974, Validation Loss = 2.4827, Validation Accuracy = 0.5200
Early stopping triggered at epoch 49
Finished training after 49 epochs!


Test acc: 0.5331530570983887


### 8e. Visualize the predictions made by ResNet-18 on the CIFAR-100 test set.

In the cell below, use your trained ResNet-18 to get the predicted classes of thr 1st 225 test set images.

Run the code below to create a 15x15 grid of CIFAR-100 images with the true and predicted classes in the title. The predicted classes are color-coded  blue if they are correct, red if they are incorrect.

In [0]:
_,_,_,_, x100_test_vis, y100_test_vis, classnames = get_dataset('cifar100', standardize_ds=False)

panel_sz = 4
grid = 15
fig, axes, = plt.subplots(nrows=grid, ncols=grid, figsize=(grid*panel_sz, grid*panel_sz))

for r in range(grid):
    for c in range(grid):
        ind = grid*r + c
        axes[r,c].imshow(x100_test_vis[ind])
        axes[r,c].set_xticks([])
        axes[r,c].set_yticks([])
        title = f'{classnames[y100_test[ind]]}\nPredicted: '
        title += f'{classnames[y_pred[ind]]}'

        color = 'blue'
        if y100_test[ind] != y_pred[ind]:
            color = 'red'

        axes[r,c].set_title(title, color=color)
plt.tight_layout()
plt.show()


### 8f. Questions

**Question 5:** Take a look at the above montage. Does the mistakes made by ResNet-18 seem reasonable? Provide some specific examples to support your conclusion.

**Answer 5:**

## Extensions

### General guidelines

1. Never integrate extensions into your base project so that they change the expected behavior of core functions. If your extension changes the core design/behavior, no problem, duplicate your working base project and add features from there.
2. Check the rubric to keep in mind how extensions on this project will be graded.
3. While I may consult your code and "written log" of what you did, **I am grading your extensions based on what you present in your 3-5 min video.**
3. I suggest documenting your explorations in a "log" or "lab notebook" style (i.e. documenting your thought/progression/discovery/learning process). I'm not grading your writing, so you can keep it succinct. **Whatever is most useful to you to remember what you did.** 
4. I suggest taking a hypothesis driven approach. For example "I was curious about X so I explored Y. I found Z, which was not what I expected because..., so then tried A..."
5. Make plots to help showcase your results.
6. **More is not necessarily better.** Generally, a small number of "in-depth" extensions count for more than many "shallow" extensions.

### AI guidelines

You may use AI in mostly any capacity for extensions. However, keep in mind:
1. There is no need to use AI at all!
2. You are welcome to use AI as a tool (e.g. automate something that is tedious, help you get unstuck, etc.). However, you should be coding, you should be thinking, you should be writing, you should be creating. If you are spending most (or even close to most) of your time typing into a chatbot and copy-pasting, you have probably gone too far with AI use.
3. I don't find large volumes of AI generated code/text/plots to be particularly impressive and you risk losing my interest while grading. Remember: I'm grading your extensions based on your video presentation. **More is not necessarily better.**

### Video guidelines

1. Please try to keep your video to 5 minutes (*I have other projects to grade!*). If you turn in a longer video, I make no promise that I will watch more than 5 minutes.
2. Your screen should be shared as you show me what you did. A live video of your face should also appear somewhere on the screen (e.g. picture-in-picture overlay / split screen).
3. Your partner should join you for the video and take turns talking, but, if necessary, it is fine to have one team member present during the record the video.
4. Do not simply read text from your notebook, do not read from a prepared script. I am not grading how polished your video presentation is (see extension grading criteria on rubric). 
5. I am looking for original and creative explorations sparked by your curiosity/interest/passion in a topic. This should be apparent in your video.
6. Be natural,, don't feel the need to impress me with fancy language. If it is helpful, imagine that we are talking one-on-one about your extension. Tell me what you did :)

### Extension ideas

#### 1. ResNet-34

Create and train the well-known network of the ResNet family called ResNet-34. Here is a suggested network configuration to experiment with:

```
block_units = [64, 128, 256, 512]
num_blocks = [3, 4, 6, 3]
first_block_strides = [1, 2, 2, 2]
```

#### 2. ResNet-50

Create and train the well-known network of the ResNet family called ResNet-50. Given its depth, it uses a "Bottleneck block" rather than a normal Residual Block, but the overall structure is very similar. Here is a suggested network configuration to experiment with:

```
block_units = [64, 128, 256, 512]
num_blocks = [3, 4, 6, 3]
first_block_strides = [1, 2, 2, 2]
```

#### 3. VGG networks on CIFAR-100

How does one or more of your VGG networks do at classifying images in CIFAR-100?

#### 4. Other ResNets on CIFAR-10

How do the other ResNets do at classifying images in CIFAR-10?

#### 5. Multi-network comparison

Compare the accuracy, efficiency, etc of any number of networks from the VGG, Inception Net, and ResNet families.

#### 6. Add support for saving/loading network weights

A key limitation of your current deep learning library is that parameters that capture the learning in networks are completely reset/lost/wiped out when the notebook kernel is terminated. Add (and test!) support for saving network parameters to disk after (or periodically during) training. Add (and test!) support for loading network parameters back into the network from disk before training. 

Be careful to include the moving mean and standard deviation parameters in batch normalization layers otherwise the whole net will not work!

#### 7. Other image datasets

Apply any of the three deep network families to another dataset of your choice. 

#### 8. Hyperparameter tuning

Try and find hyperparameters that allow Inception Net and the ResNets to achieve better accuracy on CIFAR-10 and/or CIFAR-100.

#### 9. Build other Inception Nets

We only built a single network, but just like VGG and ResNet, you can modify the network depth while following the computational motifs of the Inception Net architecture. Design and experiment with your own Inception Net!

#### 10. Analyze errors made by one or more of the nets

Make a confusion matrix for CIFAR-10 or CIFAR-100 (*a challenge to make it useful!*).

Visualize the predictions made by Inception Net and/or a VGG net, perhaps similar what was done with the ResNet.